# How to troubleshoot a diverged FDTD simulation

Tidy3D uses the Finite-Difference Time-Domain (FDTD) method, which is a popular technique for rigorously solving Maxwell's equations. However, like all numerical methods, it can sometimes diverge if not properly set up. An FDTD simulation can diverge due to various reasons. In this article, we discuss common FDTD setting issues that could potentially lead to simulation divergence. If your simulation diverged, please follow this article and perform thorough troubleshooting, which will likely resolve the issue and ensure that your next FDTD run is stable.

## Structures Inserted into PML at an Angle

Perfectly matched layer (PML) is the most commonly used [boundary condition](https://www.flexcompute.com/tidy3d/examples/notebooks/BoundaryConditions/) in FDTD simulations to truncate a simulation domain and absorb outgoing radiation. However, many divergence issues are associated with the use of PML. One of the most common causes of a diverged simulation is structures inserted into PML at an angle. This is particularly common in simulations with photonic waveguides, where PML intersects a [waveguide bend](https://www.flexcompute.com/tidy3d/examples/notebooks/EulerWaveguideBend/) or a [waveguide taper](https://www.flexcompute.com/tidy3d/examples/notebooks/EdgeCoupler/). To ensure numerical stability, you need to make sure that structures are translationally invariant into the PML.

<img src="img/diverged-fdtd-simulation.png" alt="Structures at PML"  width="800"/>

It is not always practically possible to have structures translationally invariant into PML. For example, in a [waveguide-to-ring coupling](https://www.flexcompute.com/tidy3d/examples/notebooks/WaveguideToRingCoupling/) simulation, part of the ring will have to intersect PML. In this case, using Tidy3D’s adiabatic absorber boundary condition instead of PML is a good remedy. The absorber functions similarly to PML such that it absorbs the outgoing radiation to mimic the infinite space. However, the absorber has a slightly higher reflection and requires a bit more computation than PML but it is numerically much more stable. For the demonstration, please refer to the waveguide-to-ring coupling tutorial.

In principle, you can manually extend the bent waveguide or ring into PML in a translational invariant way. This could be effective in preventing divergence but the artificial kink will inevitably lead to undesired reflection. In general, we recommend using the absorber boundary rather than this approach.

## Dispersive Material into PML

Incorporating a dispersive material into PML can also cause simulation divergence in certain scenarios. If your simulation lacks any structures inserted into PML at an angle but includes dispersive material in PML, it is advisable to substitute nondispersive material for the dispersive material. Alternatively, if dispersion is necessary, switching PML to absorber can effectively address the issue.

## Evanescent Field Leaks into PML

PML can effectively absorb outgoing radiation with minimum reflection as if the radiation just propagates into the free space. However, it's important to keep in mind that PML only absorbs propagating fields. For evanescent fields, PML can act as an amplification medium and cause a simulation to diverge. In Tidy3D, a warning will appear if the distance between a structure is smaller than half of a wavelength to prevent evanescent fields from leaking into PML. In most cases, the evanescent field will naturally die off within half a wavelength, but in some instances, a larger distance may be required. One example is when using periodic or Bloch boundary conditions in two dimensions and PML in the last dimension only. In such simulations, there could be quasi-guided modes in the periodic directions which have very long evanescent tails in the PML direction. If a simulation diverges and you suspect that evanescent fields may be leaking into PML, simply increase the simulation domain size to avoid this issue.

<img src="img/diverged-fdtd-simulation1.png" alt="Distance from PML"  width="800"/>

Additionally, sources like PointDipole, UniformCurrentSource, or CustomFieldSource can inject evanescent fields, so it's important to leave enough space between them and PML.

## Gain Medium from Fitting

When defining a dispersive material using an external fitter or Tidy3D's regular fitter, it is crucial to ensure that the fit is passive. Although the material may appear passive within the simulation frequency range, the fitting process could result in a gain medium outside of the frequency range, leading to simulation divergence. To avoid this, Tidy3D offers a [FastDispersionFitter](../_autosummary/tidy3d.plugins.dispersion.FastDispersionFitter.html) that enforces passive fitting across all frequencies. It is highly recommended to use this fitter for dispersive medium fitting.

## Courant Factor is Too Large

When conducting FDTD simulations, it's important to satisfy the Courant factor condition. This condition, also known as the Courant number or Courant-Friedrichs-Lewy (CFL) condition, is a numerical requirement that relates the time step (Δt) to the spatial step (Δx), and sometimes to the wave propagation speed (c) in the system. The Courant factor can be determined by the formula: C = c \* Δt / Δx. To ensure the stability of the numerical solution, the Courant factor must be equal to or less than 1, according to the CFL condition: C ≤ 1. By satisfying this condition, the simulation can accurately capture the wave propagation in the system, as information cannot travel further than one spatial step in one time step. Violating the CFL condition can cause the simulation to diverge and become unstable. Therefore, it's crucial to choose appropriate time and spatial step sizes for any FDTD simulation.

Tidy3D uses a default Courant factor of 0.99. When a dispersive material with `eps_inf < 1` is used, the Courant factor will be automatically adjusted to be smaller than `sqrt(eps_inf)`to ensure stability. If your simulation still diverges despite addressing any other issues discussed above, reducing the Courant factor may help.

## Additional Notes on Absorber

As discussed above, using absorber boundary is often a good remedy to resolve divergence issues related to PML. The adiabatic absorber is a multilayer system with gradually increasing conductivity. As briefly discussed above, the absorber usually has a larger undesired reflection compared to PML. In practice, this small difference rarely matters, but is important to understand for simulations that require high accuracy. There are two possible sources for the reflection from absorbers. The first, and more common one, is that the ramping up of the conductivity is not sufficiently slow, which can be remedied by increasing the number of absorber layers (40 by default). The second one is that the absorption is not high enough, such that the light reaches the PEC boundary at the end of the Absorber, travels back through it, and is still not fully attenuated before re-entering the simulation region. If this is the case, increasing the maximum conductivity (see the API reference) can help. In both cases, changing the order of the scaling of the conductivity (sigma_order) can also have an effect, but this is a more advanced setting that we typically do not recommend modifying.

## Contact Tidy3D Support

If the solutions provided in this article did not resolve your simulation issues, please contact [Tidy3D Support](https://www.flexcompute.com/tidy3d/technical-support/?site=kb&_gl=1*1i6q8q0*_ga*MTg1NzUzODgyOS4xNjg1MDgyNDQz*_ga_VB7XZB03TL*MTY5MDg2NDAxOC4zNC4xLjE2OTA4NjQzODAuNjAuMC4w). Our experienced support engineers will assist you in resolving the problems with your simulation settings.